In [1]:
import numpy as np
import pandas as pd

In [2]:
tomtom = pd.read_csv("./tomtom_conv1_CisTarget_t1_allr/filter_anno.csv", index_col=None)[["Query_ID", "Target_ID", "gene_name"]]
JSD = pd.read_csv("../../../0_JSD/Human_TF_celltype.network_3.txt", sep='\t')
JSD["Celltype"] = JSD["Celltype"].apply(lambda x: x.split("_")[1]+"_"+x.split("_")[0])
JSD.head()

,Gene,Celltype,JSDR_scale
0,ARID5A,Human11_Endothelial,3.009388
1,BCL6B,Human11_Endothelial,7.462829
2,ELK3,Human11_Endothelial,4.912828
3,EPAS1,Human11_Endothelial,3.139412
4,ERG,Human11_Endothelial,7.341058


In [3]:
bin_influ_df = pd.read_csv("./Influence/positive_influence_celltype.csv", index_col=0)
bin_influ_df=bin_influ_df[bin_influ_df.Influe > 0.5]
# bin_influ_df=bin_influ_df[bin_influ_df.Influe > np.quantile(bin_influ_df.Influe, 0.8)]
bin_influ_df.head()

,Celltype,Motif,Influe
6,Human16_Epithelial,Motif_0,1.226223
7,Human17_Epithelial,Motif_0,0.782316
14,Human23_Germ,Motif_0,0.646317
15,Human24_Epithelial,Motif_0,0.544281
18,Human29_Immune,Motif_0,0.814684


In [4]:
Motif_tf_celltype = bin_influ_df.merge(tomtom, left_on="Motif", right_on="Query_ID", how='left')
Motif_tf_celltype.head()

,Celltype,Motif,Influe,Query_ID,Target_ID,gene_name
0,Human16_Epithelial,Motif_0,1.226223,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,FOXP2
1,Human16_Epithelial,Motif_0,1.226223,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,ZNF358
2,Human16_Epithelial,Motif_0,1.226223,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,ZNF362
3,Human16_Epithelial,Motif_0,1.226223,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,ZNF384
4,Human16_Epithelial,Motif_0,1.226223,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,ZNF768


In [5]:
Motif_tf_celltype_influence_JSD = Motif_tf_celltype.merge(JSD, left_on=["gene_name","Celltype"], right_on=["Gene","Celltype"])
Motif_tf_celltype_influence_JSD.head()

,Celltype,Motif,Influe,Query_ID,Target_ID,gene_name,Gene,JSDR_scale
0,Human16_Epithelial,Motif_0,1.226223,Motif_0,tfdimers-MD00464,PDX1,PDX1,3.069992
1,Human35_Epithelial,Motif_0,0.898254,Motif_0,taipale-GTAAAYAAACA-Foxc1-DBD,FOXI1,FOXI1,17.075274
2,Human41_Epithelial,Motif_0,1.002120,Motif_0,taipale-GTAAAYAAACA-Foxc1-DBD,FOXI1,FOXI1,3.513347
3,Human74_Stromal,Motif_0,1.877806,Motif_0,taipale-GTAAAYAAACA-Foxc1-DBD,FOXC2,FOXC2,5.526845
4,Human74_Stromal,Motif_103,1.286406,Motif_103,taipale-GTAAAYAAACA-Foxc1-DBD,FOXC2,FOXC2,5.526845


In [6]:
Motif_tf_celltype_influence_JSD.to_csv("./Influence/Motif_tf_celltype_influence_JSD.csv")

In [7]:
Motif_tf_celltype_influence_JSD[["Celltype", "Motif", "Gene"]].drop_duplicates().to_csv("./Influence/Motif_tf_celltype_influence_JSD_nonDup.csv")

In [8]:
bin_influ_df = pd.read_csv("./Influence/combination_positive_influence_celltype.csv", index_col=0)
bin_influ_df=bin_influ_df[bin_influ_df.Influe > 1.5]

bin_influ_df["Motif1"] = bin_influ_df.Motif.map(lambda x: "Motif_"+str(x.split(', ')[0].strip("(")))
bin_influ_df["Motif2"] = bin_influ_df.Motif.map(lambda x: "Motif_"+str(x.split(', ')[1].strip(")")))

print(bin_influ_df.shape)
bin_influ_df.head()

,Celltype,Motif,Influe,Motif1,Motif2
22,Human32_Epithelial,"(0, 1)",1.600425,Motif_0,Motif_1
25,Human36_Erythroid,"(0, 1)",1.791706,Motif_0,Motif_1
39,Human4_Erythroid,"(0, 1)",1.704349,Motif_0,Motif_1
47,Human57_Erythroid,"(0, 1)",1.938517,Motif_0,Motif_1
51,Human60_Immune,"(0, 1)",1.673119,Motif_0,Motif_1


In [9]:
Motif_tf_celltype = bin_influ_df.merge(tomtom, right_on="Query_ID", left_on="Motif1", how='inner')
Motif_tf_celltype.rename(columns={"Query_ID":"Query_ID_M1", "Target_ID":"Target_ID_M1", "gene_name":"gene_name_M1"}, inplace=True)
Motif_tf_celltype.head()

,Celltype,Motif,Influe,Motif1,Motif2,Query_ID_M1,Target_ID_M1,gene_name_M1
0,Human32_Epithelial,"(0, 1)",1.600425,Motif_0,Motif_1,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,FOXP2
1,Human32_Epithelial,"(0, 1)",1.600425,Motif_0,Motif_1,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,ZNF358
2,Human32_Epithelial,"(0, 1)",1.600425,Motif_0,Motif_1,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,ZNF362
3,Human32_Epithelial,"(0, 1)",1.600425,Motif_0,Motif_1,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,ZNF384
4,Human32_Epithelial,"(0, 1)",1.600425,Motif_0,Motif_1,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,ZNF768


In [10]:
Motif_tf_celltype = Motif_tf_celltype.merge(tomtom, right_on="Query_ID", left_on="Motif2", how='inner')
Motif_tf_celltype.rename(columns={"Query_ID":"Query_ID_M2", "Target_ID":"Target_ID_M2", "gene_name":"gene_name_M2"}, inplace=True)
Motif_tf_celltype.head()

,Celltype,Motif,Influe,Motif1,Motif2,Query_ID_M1,Target_ID_M1,gene_name_M1,Query_ID_M2,Target_ID_M2,gene_name_M2
0,Human32_Epithelial,"(0, 4)",1.772996,Motif_0,Motif_4,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,FOXP2,Motif_4,tfdimers-MD00492,ZEB1
1,Human32_Epithelial,"(0, 4)",1.772996,Motif_0,Motif_4,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,FOXP2,Motif_4,tfdimers-MD00492,ZNF354C
2,Human32_Epithelial,"(0, 4)",1.772996,Motif_0,Motif_4,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,ZNF358,Motif_4,tfdimers-MD00492,ZEB1
3,Human32_Epithelial,"(0, 4)",1.772996,Motif_0,Motif_4,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,ZNF358,Motif_4,tfdimers-MD00492,ZNF354C
4,Human32_Epithelial,"(0, 4)",1.772996,Motif_0,Motif_4,Motif_0,flyfactorsurvey-sqz_SOLEXA_5_FBgn0010768,ZNF362,Motif_4,tfdimers-MD00492,ZEB1


In [11]:
Motif_tf_celltype_influence_JSD = Motif_tf_celltype.merge(JSD, left_on=["gene_name_M1","Celltype"], right_on=["Gene","Celltype"])
Motif_tf_celltype_influence_JSD.rename(columns={"Gene":"Gene_M1", "JSDR_scale":"JSDR_scale_M1"}, inplace=True)
Motif_tf_celltype_influence_JSD.head()

,Celltype,Motif,Influe,Motif1,Motif2,Query_ID_M1,Target_ID_M1,gene_name_M1,Query_ID_M2,Target_ID_M2,gene_name_M2,Gene_M1,JSDR_scale_M1
0,Human74_Stromal,"(0, 8)",1.697204,Motif_0,Motif_8,Motif_0,taipale-GTAAAYAAACA-Foxc1-DBD,FOXC2,Motif_8,flyfactorsurvey-CG2052_SOLEXA_2.5_FBgn0039905,FOXP2,FOXC2,5.526845
1,Human74_Stromal,"(0, 8)",1.697204,Motif_0,Motif_8,Motif_0,taipale-GTAAAYAAACA-Foxc1-DBD,FOXC2,Motif_8,flyfactorsurvey-CG2052_SOLEXA_2.5_FBgn0039905,ZNF358,FOXC2,5.526845
2,Human74_Stromal,"(0, 8)",1.697204,Motif_0,Motif_8,Motif_0,taipale-GTAAAYAAACA-Foxc1-DBD,FOXC2,Motif_8,flyfactorsurvey-CG2052_SOLEXA_2.5_FBgn0039905,ZNF362,FOXC2,5.526845
3,Human74_Stromal,"(0, 8)",1.697204,Motif_0,Motif_8,Motif_0,taipale-GTAAAYAAACA-Foxc1-DBD,FOXC2,Motif_8,flyfactorsurvey-CG2052_SOLEXA_2.5_FBgn0039905,ZNF384,FOXC2,5.526845
4,Human74_Stromal,"(0, 8)",1.697204,Motif_0,Motif_8,Motif_0,taipale-GTAAAYAAACA-Foxc1-DBD,FOXC2,Motif_8,flyfactorsurvey-CG2052_SOLEXA_2.5_FBgn0039905,ZNF768,FOXC2,5.526845


In [12]:
Motif_tf_celltype_influence_JSD = Motif_tf_celltype_influence_JSD.merge(JSD, left_on=["gene_name_M2","Celltype"], right_on=["Gene","Celltype"])
Motif_tf_celltype_influence_JSD.rename(columns={"Gene":"Gene_M2", "JSDR_scale":"JSDR_scale_M2"}, inplace=True)
Motif_tf_celltype_influence_JSD

,Celltype,Motif,Influe,Motif1,Motif2,Query_ID_M1,Target_ID_M1,gene_name_M1,Query_ID_M2,Target_ID_M2,gene_name_M2,Gene_M1,JSDR_scale_M1,Gene_M2,JSDR_scale_M2
0,Human74_Stromal,"(0, 110)",1.673760,Motif_0,Motif_110,Motif_0,taipale-GTAAAYAAACA-Foxc1-DBD,FOXC2,Motif_110,taipale-NNGNNACGCCCMYTTTNN-KLF13-full,SP7,FOXC2,5.526845,SP7,27.429536
1,Human74_Stromal,"(0, 110)",1.673760,Motif_0,Motif_110,Motif_0,taipale-GTAAAYAAACA-Foxc1-DBD,FOXC2,Motif_110,flyfactorsurvey-CG3065_F1-3_SOLEXA_2.5_FBgn003...,SP7,FOXC2,5.526845,SP7,27.429536
2,Human74_Stromal,"(0, 110)",1.673760,Motif_0,Motif_110,Motif_0,taipale-GTAAAYAAACA-Foxc1-DBD,FOXC2,Motif_110,flyfactorsurvey-CG3065_F1-5_SOLEXA_2.5_FBgn003...,SP7,FOXC2,5.526845,SP7,27.429536
3,Human74_Stromal,"(0, 110)",1.673760,Motif_0,Motif_110,Motif_0,taipale-GTAAAYAAACA-Foxc1-DBD,FOXC2,Motif_110,flyfactorsurvey-CG5669_SOLEXA_5_FBgn0039169,SP7,FOXC2,5.526845,SP7,27.429536
4,Human74_Stromal,"(0, 110)",1.673760,Motif_0,Motif_110,Motif_0,taipale-GTAAAYAAACA-Foxc1-DBD,FOXC2,Motif_110,taipale-GCCMCGCCCMC-KLF16-DBD,SP7,FOXC2,5.526845,SP7,27.429536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3386,Human91_Immune,"(37, 109)",1.577602,Motif_37,Motif_109,Motif_37,tfdimers-MD00095,NEUROD1,Motif_109,taipale-AAAAAGCGGAAGTN-SPI1-full,SPI1,NEUROD1,3.488112,SPI1,3.624920
3387,Human91_Immune,"(37, 109)",1.577602,Motif_37,Motif_109,Motif_37,tfdimers-MD00095,NEUROD1,Motif_109,tfdimers-MD00480,TAL1,NEUROD1,3.488112,TAL1,8.046382
3388,Human85_Stromal,"(55, 67)",1.692585,Motif_55,Motif_67,Motif_55,taipale-NNGNNACGCCCMYTTTNN-KLF13-full,KLF16,Motif_67,jaspar-MA0353.1,NFATC2,KLF16,4.431596,NFATC2,6.823253
3389,Human85_Stromal,"(55, 67)",1.692585,Motif_55,Motif_67,Motif_55,flyfactorsurvey-crol-F7-16_SOLEXA_FBgn0020309,KLF16,Motif_67,jaspar-MA0353.1,NFATC2,KLF16,4.431596,NFATC2,6.823253


In [13]:
Motif_tf_celltype_influence_JSD.to_csv("./Influence/Combination_Motif_tf_celltype_influence_JSD.csv")

In [14]:
Motif_tf_celltype_influence_JSD[["Celltype", "Motif", "gene_name_M1", "gene_name_M2"]].drop_duplicates().to_csv("./Influence/Combination_Motif_tf_celltype_influence_JSD_nonDup.csv")